In [1]:
import boto3

In [2]:
def iterate_bucket(bucket):
    client = boto3.client('s3')
    paginator = client.get_paginator('list_objects_v2')
    page_iterator = paginator.paginate(Bucket=bucket)
    
    for page in page_iterator:
        if page['KeyCount'] > 0:
            for item in page['Contents']:
                yield item

In [10]:
counter=0
pathlist=[]
#for i in iterate_bucket(bucket='millionsongproject'):
for i in iterate_bucket(bucket='cosc502millionsong'):
    if counter<200000:
        if i['Key'].endswith('.h5') and not i['Key']=='millionsongsubset/MillionSongSubset/AdditionalFiles/subset_msd_summary_file.h5':
            pathlist.append(i['Key'])
            counter+=1
    else:
        break

In [32]:
pathlist[0]

'millionsongsubset/MillionSongSubset/data/A/A/A/TRAAAAW128F429D538.h5'

In [5]:
import findspark
findspark.init()
from pyspark import SparkContext
from pyspark.sql import SparkSession
sc = SparkContext()
ss=SparkSession \
.builder \
.getOrCreate()

In [12]:
path_rdd=sc.parallelize(pathlist)

In [25]:
path_rdd.take(5)

['A/A/A/TRAAAAK128F9318786.h5',
 'A/A/A/TRAAAAV128F421A322.h5',
 'A/A/A/TRAAAAW128F429D538.h5',
 'A/A/A/TRAAAAY128F42A73F0.h5',
 'A/A/A/TRAAABD128F429CF47.h5']

In [13]:
import pandas as pd
import numpy as np

In [21]:
path_iter=path_rdd.toLocalIterator()

In [22]:
next(path_iter)

'millionsongsubset/MillionSongSubset/data/A/A/A/TRAAAAW128F429D538.h5'

In [16]:
artist_terms=[]
artist_terms_freq=[]
artist_terms_weight=[]
similar_artists=[]
songs = []
bars_confidence=[]
bars_start=[]
beats_confidence=[]
beats_start=[]
sections_confidence=[]
sections_start=[]
segments_confidence=[]
segments_loudness_max=[]
segments_loudness_max_time=[]
segments_loudness_start=[]
segments_pitches=[]
segments_start=[]
segments_timbre=[]
tatums_confidence=[]
tatums_start=[]
analysis_songs=[]
artist_mbtags=[]
artist_mbtags_count=[]
musicbrainz_songs=[]

In [34]:
import h5py
import s3fs
s3 = s3fs.S3FileSystem()
BUCKET='s3://cosc502millionsong/'
for a in path_iter:
    if a=='millionsongsubset/MillionSongSubset/data/A/A/A/TRAAAAW128F429D538.h5':
        continue
    address=BUCKET+a
    with h5py.File(s3.open(address, 'rb'), 'r', lib_version='labtest') as f:
        """
        metadata: artist_terms, 'artist_terms_freq', 'artist_terms_weight', 'similar_artists', 'songs'
        """
        data=f['metadata']
        artist_terms.append(data['artist_terms'][:])
        artist_terms_freq.append(data['artist_terms_freq'][:])
        artist_terms_weight.append(data['artist_terms_weight'][:])
        similar_artists.append(data['similar_artists'][:])
        songs.append(data['songs'][:])
        """
        analysis: 'bars_confidence', 'bars_start', 'beats_confidence', 
        'beats_start', 'sections_confidence', 'sections_start', 
        'segments_confidence', 'segments_loudness_max', 'segments_loudness_max_time',
        'segments_loudness_start', 'segments_pitches', 'segments_start', 
        'segments_timbre', 'songs', 'tatums_confidence', 'tatums_start'
        """
        data=f['analysis']
        bars_confidence.append(data['bars_confidence'][:])
        bars_start.append(data['bars_start'][:])
        beats_confidence.append(data['beats_confidence'][:])
        beats_start.append(data['beats_start'][:])
        analysis_songs.append(data['songs'][:])
        sections_confidence.append(data['sections_confidence'][:])
        sections_start.append(data['sections_start'][:])
        segments_confidence.append(data['segments_confidence'][:])
        segments_loudness_max.append(data['segments_loudness_max'][:])
        segments_loudness_max_time.append(data['segments_loudness_max_time'][:])
        segments_loudness_start.append(data['segments_loudness_start'][:])
        segments_pitches.append(data['segments_pitches'][:])
        segments_start.append(data['segments_start'][:])
        segments_timbre.append(data['segments_timbre'][:])
        tatums_confidence.append(data['tatums_confidence'][:])
        tatums_start.append(data['tatums_start'][:])
        """
        musicbrainz: 'artist_mbtags', 'artist_mbtags_count', 'songs'
        """
        data=f['musicbrainz']
        artist_mbtags.append(data['artist_mbtags'][:])
        artist_mbtags_count.append(data['artist_mbtags_count'][:])
        musicbrainz_songs.append(data['songs'][:])


In [52]:
type(artist_terms[0])

numpy.ndarray

In [60]:
df = pd.DataFrame()
df['artist_terms']=pd.Series(artist_terms)
df['artist_terms_freq']=pd.Series(artist_terms_freq)
df['artist_terms_weight']=pd.Series(artist_terms_weight)
df['similar_artists']=pd.Series(similar_artists)
df['songs']=pd.Series(songs)
df['bars_confidence']=pd.Series(bars_confidence)
df['bars_start']=pd.Series(bars_start)
df['beats_confidence']=pd.Series(beats_confidence)
df['beats_start']=pd.Series(beats_start)
df['analysis_songs']=pd.Series(analysis_songs)
df['sections_confidence']=pd.Series(sections_confidence)
df['sections_start']=pd.Series(sections_start)
df['segments_confidence']=pd.Series(segments_confidence)
df['segments_loudness_max']=pd.Series(segments_loudness_max)
df['segments_loudness_max_time']=pd.Series(segments_loudness_max_time)
df['segments_loudness_start']=pd.Series(segments_loudness_start)
df['segments_pitches']=pd.Series(segments_pitches)
df['segments_start']=pd.Series(segments_start)
df['segments_timbre']=pd.Series(segments_timbre)
df['tatums_confidence']=pd.Series(tatums_confidence)
df['tatums_start']=pd.Series(tatums_start)
df['artist_mbtags']=pd.Series(artist_mbtags)
df['artist_mbtags_count']=pd.Series(artist_mbtags_count)
df['musicbrainz_songs']=pd.Series(musicbrainz_songs)

In [62]:
df.head(5)

,artist_terms,artist_terms_freq,artist_terms_weight,similar_artists,songs,bars_confidence,bars_start,beats_confidence,beats_start,analysis_songs,...,segments_loudness_max_time,segments_loudness_start,segments_pitches,segments_start,segments_timbre,tatums_confidence,tatums_start,artist_mbtags,artist_mbtags_count,musicbrainz_songs
0,"[b'salsa', b'cumbia', b'tejano', b'ranchera', ...","[1.0, 0.9422390717941641, 0.9422390717941641, ...","[1.0, 0.9582578450180738, 0.9582578450180738, ...","[b'ARFSJUG11C8A421AAD', b'AR8SD041187FB36015',...","[[b'', 290021, 0.4873567909281477, 0.343428378...","[0.98, 0.399, 0.185, 0.27, 0.422, 0.0, 0.445, ...","[0.73152, 1.39732, 2.04852, 2.68691, 3.315, 3....","[0.98, 0.399, 0.185, 0.27, 0.422, 0.0, 0.445, ...","[0.73152, 1.39732, 2.04852, 2.68691, 3.315, 3....","[[22050, b'fa329738005ca53715d9f7381a0d1fe3', ...",...,"[0.27572, 0.1589, 0.0515, 0.0741, 0.09185, 0.0...","[-60.0, -59.9, -12.744, -12.003, -12.991, -15....","[[1.0, 0.911, 0.18, 0.334, 0.327, 0.344, 0.302...","[0.0, 0.28154, 0.48395, 0.6937, 0.97859, 1.361...","[[0.0, 171.124, 9.459, -28.489, 57.482, -50.06...","[0.482, 0.676, 0.627, 0.549, 0.279, 0.264, 0.2...","[0.42132, 0.73152, 1.06609, 1.39732, 1.72854, ...",[],[],"[[0, 0]]"
1,"[b'pop rock', b'new wave', b'dance rock', b'ro...","[0.9885838625154639, 0.9672504640243684, 0.820...","[1.0, 0.9636972066614938, 0.9267729972686404, ...","[b'AR4R0741187FB39AF2', b'AR0D7K21187B9AD14E',...","[[b'', 19072, 0.6303823341467806, 0.4542311565...","[0.017, 0.05, 0.014, 0.008, 0.114, 0.019, 0.08...","[1.30621, 3.29887, 5.30252, 7.32327, 9.33775, ...","[0.809, 0.616, 0.789, 0.66, 0.439, 0.758, 0.60...","[0.81002, 1.30621, 1.80617, 2.2996, 2.80049, 3...","[[22050, b'43cd1abd45d5a2dda16a3c65b4963bd4', ...",...,"[0.34385, 0.07741, 0.04658, 0.07981, 0.04477, ...","[-60.0, -27.665, -21.241, -15.222, -18.915, -1...","[[0.651, 0.592, 0.647, 0.494, 0.683, 0.919, 1....","[0.0, 0.70517, 1.03052, 1.21052, 1.52404, 1.72...","[[24.937, 37.465, 177.22, -216.443, 56.3, 202....","[0.601, 0.556, 0.523, 0.49, 0.466, 0.44, 0.428...","[0.56254, 0.81002, 1.05749, 1.30621, 1.55494, ...","[b'uk', b'british', b'english']","[1, 1, 1]","[[0, 1982]]"
2,"[b'pop punk', b'ska punk', b'breakcore', b'alt...","[0.8872883012773332, 0.7902037283563664, 0.790...","[1.0, 0.9609057433767874, 0.9592366232787087, ...","[b'ARUA62A1187B99D9B0', b'ARHJFFY1187B98BA76',...","[[b'', 30973, 0.6510456608317947, 0.4017236855...","[0.175, 0.409, 0.639, 0.067, 0.016, 0.066, 0.0...","[1.06368, 2.91491, 4.76729, 6.61852, 8.46978, ...","[0.883, 0.738, 0.484, 0.609, 0.625, 0.719, 0.4...","[0.13576, 0.59914, 1.06368, 1.52591, 1.99045, ...","[[22050, b'580a8fe08ef0f1c7734b84547d7a8bc7', ...",...,"[0.06094, 0.06433, 0.02255, 0.02018, 0.02463, ...","[-60.0, -59.828, -19.551, -32.609, -21.899, -2...","[[1.0, 0.529, 0.407, 0.423, 0.524, 0.509, 0.65...","[0.0, 0.06603, 0.24395, 0.57034, 0.92567, 1.26...","[[0.089, 169.621, 5.435, -30.061, 54.144, -50....","[1.0, 0.98, 0.932, 0.87, 0.82, 0.793, 0.768, 0...","[0.13576, 0.36918, 0.59914, 0.83141, 1.06368, ...",[],[],"[[0, 2007]]"
3,"[b'southern gospel', b'bluegrass', b'gospel', ...","[1.0, 0.8776784423307522, 0.5756383007306756, ...","[1.0, 0.889349058121347, 0.660827892217304, 0....","[b'ARHNMEZ11F50C4706C', b'ARCAWRI1187B98CDA6',...","[[b'', 432935, 0.5352927355118197, 0.385470550...","[0.121, 0.511, 0.356, 0.397, 0.193, 0.262, 0.3...","[1.17118, 2.44699, 3.76552, 5.07403, 6.38454, ...","[0.438, 0.164, 0.143, 0.044, 0.047, 0.22, 0.27...","[0.74856, 1.17118, 1.59278, 2.0154, 2.44699, 2...","[[22050, b'8ee90e90bb8714300574486f379effb5', ...",...,"[0.43155, 0.15939, 0.13164, 0.08969, 0.11711, ...","[-60.0, -58.929, -54.569, -48.97, -46.964, -45...","[[0.778, 0.152, 0.101, 0.119, 0.145, 1.0, 0.14...","[0.0, 0.44975, 0.62413, 1.06567, 1.27497, 1.54...","[[0.047, 170.004, 9.863, -29.763, 58.312, -50....","[0.136, 0.127, 0.113, 0.112, 0.104, 0.09, 0.07...","[0.53929, 0.74856, 0.95987, 1.17118, 1.38249, ...",[],[],"[[0, 

In [63]:
from io import StringIO

csv_buffer = StringIO()
df.to_csv(csv_buffer)
s3_resource = boto3.resource('s3')
s3_resource.Object("cosc502millionsong", 'subset_test.csv').put(Body=csv_buffer.getvalue())

{'ResponseMetadata': {'RequestId': '19E1732382C52357',
  'HostId': 'Pp2Bu/J4udlt3qdw549FM+BkkyRyADGiiILmkrYm5BJyeOjuQC9plwMrNLlzFyHlRJEg2b1f1Xg=',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': 'Pp2Bu/J4udlt3qdw549FM+BkkyRyADGiiILmkrYm5BJyeOjuQC9plwMrNLlzFyHlRJEg2b1f1Xg=',
   'x-amz-request-id': '19E1732382C52357',
   'date': 'Tue, 21 Apr 2020 18:00:34 GMT',
   'etag': '"49baac1a99e4f8c94f6868dfc29e61f6"',
   'content-length': '0',
   'server': 'AmazonS3'},
  'RetryAttempts': 0},
 'ETag': '"49baac1a99e4f8c94f6868dfc29e61f6"'}

In [70]:
bytes_to_write = df.to_csv(None).encode()
fs = s3fs.S3FileSystem()
with fs.open('s3://cosc502millionsong/test_file.csv', 'wb') as f:
    f.write(bytes_to_write)

In [75]:
df.to_csv('s3://cosc502millionsong/df_test.csv', index=True)

In [25]:
import h5py
import s3fs

s3 = s3fs.S3FileSystem()

with h5py.File(s3.open('s3://millionsongproject/A/A/A/TRAAAAV128F421A322.h5', 'rb'), 'r', lib_version='labtest') as f:
    file_keys = list(f.keys())
    print(file_keys)

AttributeError: 'S3File' object has no attribute 'seek'

In [20]:
import h5py
import s3fs

s3 = s3fs.S3FileSystem()

with h5py.File(s3.open('s3://cosc502millionsong/millionsongsubset/MillionSongSubset/data/A/A/A/TRAAAAW128F429D538.h5', 'rb'), 'r', lib_version='labtest') as f:
    file_keys = list(f.keys())
    print(file_keys)

['analysis', 'metadata', 'musicbrainz']


In [76]:
test_file = ss.read.csv('s3://cosc502millionsong/df_test.csv')

In [77]:
test_file.schema

StructType(List(StructField(_c0,StringType,true),StructField(_c1,StringType,true),StructField(_c2,StringType,true),StructField(_c3,StringType,true),StructField(_c4,StringType,true),StructField(_c5,StringType,true),StructField(_c6,StringType,true),StructField(_c7,StringType,true),StructField(_c8,StringType,true),StructField(_c9,StringType,true),StructField(_c10,StringType,true),StructField(_c11,StringType,true),StructField(_c12,StringType,true),StructField(_c13,StringType,true),StructField(_c14,StringType,true),StructField(_c15,StringType,true),StructField(_c16,StringType,true),StructField(_c17,StringType,true),StructField(_c18,StringType,true),StructField(_c19,StringType,true),StructField(_c20,StringType,true),StructField(_c21,StringType,true),StructField(_c22,StringType,true),StructField(_c23,StringType,true),StructField(_c24,StringType,true)))

In [78]:
test_file.show(2)

+----+--------------------+-----------------+-------------------+---------------+-----+---------------+----------+----------------+-----------+--------------+-------------------+--------------+-------------------+--------------------+--------------------+--------------------+----------------+--------------+---------------+-----------------+------------+-------------+-------------------+-----------------+
| _c0|                 _c1|              _c2|                _c3|            _c4|  _c5|            _c6|       _c7|             _c8|        _c9|          _c10|               _c11|          _c12|               _c13|                _c14|                _c15|                _c16|            _c17|          _c18|           _c19|             _c20|        _c21|         _c22|               _c23|             _c24|
+----+--------------------+-----------------+-------------------+---------------+-----+---------------+----------+----------------+-----------+--------------+-------------------+------